## https://www.kaggle.com/imoore/60k-stack-overflow-questions-with-quality-rate
## We recommend you to reach above websites and click the "new notebook" inside, turn on the TPU accelerator. The Kaggle enviromnet would be preferable to run our code. If you want to run our code locally, please download the dataset from above website, and change the first few lines in code to refer to your directory with local dataset.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/60k-stack-overflow-questions-with-quality-rate/valid.csv
/kaggle/input/60k-stack-overflow-questions-with-quality-rate/train.csv


In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid
import sklearn
import warnings
warnings.filterwarnings("ignore")

In [3]:
stack_data1 = pd.read_csv(r'/kaggle/input/60k-stack-overflow-questions-with-quality-rate/train.csv')
stack_data2 = pd.read_csv(r'/kaggle/input/60k-stack-overflow-questions-with-quality-rate/valid.csv')
print(stack_data1['Y'].value_counts())
stack_data = pd.concat([stack_data1,stack_data2])
stack_data_f = stack_data.dropna(subset=['Y'])

HQ          15000
LQ_EDIT     15000
LQ_CLOSE    15000
Name: Y, dtype: int64


In [4]:
stack_data_f['text'] = stack_data_f.Title+': '+stack_data_f.Body

# if the character in text not from a-z, A-Z replace it with empty
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^(a-zA-Z)\s]','', text)
    return text
stack_data_f.text = stack_data_f.text.apply(clean_text)

In [5]:
#Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(stack_data_f.text, stack_data_f.Y, test_size=0.2, random_state=0 )

# Preparation of classifiers
classifiers = [
    LogisticRegression(C=1),
    MultinomialNB(),
    DecisionTreeClassifier(),
    RandomForestClassifier()]

# Find the best classifier as out baseline
for cls in classifiers:
    text_clf = Pipeline([
        ('vect', TfidfVectorizer(ngram_range=(1,1))),
        ('clf', cls)])
    text_clf.fit(X_train, y_train)
    print(str(cls) +': ' + str(text_clf.score(X_test, y_test)))

LogisticRegression(C=1): 0.87625
MultinomialNB(): 0.78075
DecisionTreeClassifier(): 0.7945
RandomForestClassifier(): 0.8325833333333333


**Since we discovered that the Logistic Regression has the best performance among 4 classifiers. We then use GridSearch to find the corresponding best hyperparameters.**

In [6]:
def GridSearch(cls, parameters, X, y):    
    results = pd.DataFrame()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    
    for ind,par in enumerate(list(ParameterGrid(parameters))):
        text_clf = Pipeline([
                ('vect', TfidfVectorizer()),
                ('clf', classifier(**par))])
        text_clf.fit(X_train, y_train)
        predicted = text_clf.predict(X_test)
        results.loc[str(par),'results'] = text_clf.score(X_test, y_test)
        results.loc[str(par),'parameters'] = ind
        ind_best = results.sort_values(by=['results'], ascending=False).iloc[0,1]

    return list(ParameterGrid(parameters))[int(ind_best)]

classifier = LogisticRegression
parameters = {
    'solver':['saga'],
    'C': [1, 1.5, 2],
    'penalty': ['l1', 'l2']
 }

results = GridSearch(classifier, parameters, stack_data_f.text, stack_data_f.Y)
results

{'C': 2, 'penalty': 'l1', 'solver': 'saga'}

In [7]:
classifier = LogisticRegression(**results)

text_clf = Pipeline([
                ('vect', TfidfVectorizer()),
                ('clf', classifier)])
text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)
text_clf.score(X_test, y_test)

0.8925